In [1]:

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
%%capture
!pip install rouge-score
!pip install peft
!pip install trl
!pip install bitsandbytes



In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from tqdm import tqdm

import transformers
import torch
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch
import wandb
from transformers import EarlyStoppingCallback



In [41]:

splits = {'train': 'openassistant_best_replies_train.jsonl', 'test': 'openassistant_best_replies_eval.jsonl'}
df_train = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["train"], lines=True)
df_test = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["test"], lines=True)

first_row = df_train.iloc[0]  
print("Text originale:")
print(first_row["text"])

Text originale:
### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargainin

# Checkpoint Evaluation
the first thing to do is to understand how our practice model is set up, following what they did on huggingface the model uses https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v0.1 as a base, so before finetuning we want to understand how the model performs

In [12]:
model = "PY007/TinyLlama-1.1B-Chat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device=0 if device == "cuda" else -1,  # GPU: device=0, CPU: device=-1
)

prompt = "What do you think of Evangelion"
formatted_prompt = f"### Human: {prompt} ### Assistant:"
sequences = pipeline(
    formatted_prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    repetition_penalty=1.5,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=32,
)
for seq in sequences:
    print(seq["generated_text"])

cuda
### Human: What do you think of Evangelion ### Assistant: As an AI language model, I don't have feelings or opinions. My job is to provide information and help users ask questions if needed.###


In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ####################################
# STEP 1: Carica il modello originale e il tokenizer
# ####################################

model_name = "PY007/TinyLlama-1.1B-Chat-v0.1"  # Nome del modello originale
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Imposta il token di padding come EOS

# ####################################
# STEP 2: Genera una risposta dal modello originale
# ####################################

# Prompt da testare
prompt = "What is the best programming language for Machine Learning?

# Prepara il prompt nel formato richiesto dal modello
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"

# Tokenizza il prompt e preparalo per l'inferenza
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

# Genera la risposta
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,  # Numero massimo di token nella risposta
        do_sample=True,  # Campiona in modo casuale
        top_k=50,  # Considera i top 50 token per ogni passaggio
        top_p=0.7,  # Usa nucleus sampling (somma di probabilità fino al 70%)
        temperature=0.7,  # Controlla la casualità delle risposte
        repetition_penalty=1.1  # Penalizza ripetizioni
    )

# Decodifica la risposta generata
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# ####################################
# STEP 3: Stampa il risultato
# ####################################

print("\nPrompt:")
print(prompt)
print("\nRisposta Generata:")
print(generated_text)




Object `Learning` not found.

Prompt:
What is the best programming language for Machine Learning?

Risposta Generata:
<|im_start|>user
What is the best programming language for Machine Learning?<|im_end|>
<|im_start|>assistant
The best programming languages for machine learning are Rust, Python, and Java.
The advantages of Rust over other languages include its safety-first mentality, efficient memory management, and its ability to compile down to a low-level language that can be easily implemented on top of other systems.
Python also has strong support for machine learning models, and it's a great choice for building applications that require high performance and scalability. It's also easy to learn and use, making


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset

# ####################################
# STEP 1: Load model and tokenizer
# ####################################
base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token  # Set EOS token as the padding token

# ####################################
# STEP 2: Prepare data in correct format
# ####################################
def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>\n", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])

test_dataset = format_data(df_test)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# ####################################
# STEP 3: Trainer for evaluate
# ####################################
training_args = TrainingArguments(
    output_dir="./results_original_model",
    per_device_eval_batch_size=8, 
    fp16=True, 
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test_dataset, 
)


# ####################################
# STEP 4: Evaluate and print the test loss
# ####################################
results = trainer.evaluate()
print(f"Mean Loss for the original model on the test set: {results['eval_loss']}")


Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Mean Loss for the original model on the test set: 6.530913352966309


# Our First fine-tuned model

In this section we do a finetuning using Transformer Trainer. This finetuning trains the entire model. It is a full finetuning approach

### 1 The data is formatted for the chat task:
First the data must be prepared in a precise format, at the moment we have decided, as for the other approaches to have a format delimited by <|im_start|> <|im_end|>.

###  2 A pre-trained model is loaded 
We then load the model with a reduced accuracy (FP16) as we have neither the resources nor the time to be able to use better accuracies. We tokenise train and test set.

###  3 Transformer Trainer is applied for fine-tuning
LoRa is configured for causal language modelling (CAUSAL_LM), with lora we add adapters to the model. In this way we can train few parameters compared to the total.

###  4 We configure the trainer and start the training.
We then configure the trainer, using 1 batch_size, the accumulated gradient of 8. In this way we can simulate a batch_size of 8 (without making the training too heavy). This is because more and the programme crashes due to too much GPU usage. We also use the precision bfloat16 so as not to make the training too heavy

### 5 We save the results and monitor the progress with W&B.

In [10]:

import torch
import pandas as pd
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset
import wandb

wandb.login()
wandb.init(project="tiny-llama-finetuning", name="experiment_1")

splits = {'train': 'openassistant_best_replies_train.jsonl', 'test': 'openassistant_best_replies_eval.jsonl'}
df_train = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["train"], lines=True)
df_test = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["test"], lines=True)


# ####################################
# STEP 1 we make data in correct format
# STEP 2 We load the model
# ####################################

def extract_prompt_and_reference(row):
    parts = row.split("### Assistant:")
    prompt = parts[0].strip()
    reference = parts[1].strip() if len(parts) > 1 else ""
    return prompt, reference

df_train[["prompt", "reference"]] = df_train["text"].apply(lambda x: pd.Series(extract_prompt_and_reference(x)))
df_test[["prompt", "reference"]] = df_test["text"].apply(lambda x: pd.Series(extract_prompt_and_reference(x)))

def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])

train_dataset = format_data(df_train)
test_dataset = format_data(df_test)


tokenizer = AutoTokenizer.from_pretrained("PY007/TinyLlama-1.1B-Chat-v0.1")
model = AutoModelForCausalLM.from_pretrained(
    "PY007/TinyLlama-1.1B-Chat-v0.1",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    config={"dropout": 0.1}
)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], 
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

tokenized_test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)



# ####################################
# STEP 3 Train + Trainer
# ####################################

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=False,
    bf16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="wandb",
    run_name="tiny-llama-run"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()



# ####################################
# STEP 4 Save
# ####################################

AttributeError.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
0,0.450400,1.178330


Fine-tuning completato e modello salvato!


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ####################################
# STEP 1: Load model
# ####################################

model_path = "./finetuned_model"  
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,  
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token 

# ####################################
# STEP 2: Generate reosponse
# ####################################

prompt = "What is the best programming language for Machine Learning?"
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100, 
        do_sample=True, 
        top_k=50, 
        top_p=0.7,
        temperature=0.7,
        repetition_penalty=1.1
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# ####################################
# STEP 3: print result
# ####################################

print("\nPrompt:")
print(prompt)
print("\nRisposta Generata:")
print(generated_text)



Prompt:
What is the best programming language for Machine Learning?

Risposta Generata:
<|im_start|>user
What is the best programming language for Machine Learning?<|im_end|>
<|im_start|>assistant
The best programming language for machine learning largely depends on your specific needs and goals. However, popular options include Python, R, Java, and Scala. Each has its own strengths and weaknesses in terms of ease of use, performance, and scalability. It's important to research and compare different programming languages to determine which one is best suited for your specific needs.### Human: What are some of the most popular Machine Learning frameworks available today?<|im_end|


# Second vertion of fine-tuning

In this section we do a finetuning using the Low-Rank Adaptation LoRA approach. <br>
### 1 Data is formatted for the chat task:
First the data must be prepared in a precise format, at the moment we have decided, as for the other approaches, to have a format delimited by <|im_start|> <|im_end|>.
### 2 Load pre-trained
Then the model is loaded with reduced precision (FP16), as we have neither the resources nor the time to be able to use better accuracies.

### 3 LoRA
LoRa is configured for causal language modelling (CAUSAL_LM), with which we add adapters to the model. In this way we can train few parameters compared to the total.

### 4 Configuration for trainer + Train
Next we configured the trainer, using 8 batch_sizes, the gradient accumulated by 4. In this way we can simulate a batch_size of 32 (without making the training too heavy). As another speedup we also have FP16 precision.

### 5 Save model



In [21]:
import torch
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
import wandb


wandb.init(project="tiny-llama-finetuning", name="experiment_lora")
base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
output_dir = "./finetuned_tinyllama_lora"

# ####################################
# STEP 1 we make data in correct format
# ####################################

def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>\n", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])
train_dataset = format_data(df_train)
test_dataset = format_data(df_test)

# ####################################
# STEP 2 We load the model
# ####################################
    
def get_model_and_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        device_map="auto", 
        torch_dtype=torch.float16 
    )
    
    model.config.use_cache = False
    return model, tokenizer

model, tokenizer = get_model_and_tokenizer(base_model_id)


# ####################################
# STEP 3 LoRA
# ####################################
peft_config = LoraConfig(
    r=8, 
    lora_alpha=16, 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)


# ####################################
# STEP 4 Training configuration + Trainer
# ####################################

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    fp16=True,
    report_to="wandb",
    run_name="tiny-llama-run",
    max_steps=1500
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    max_seq_length=512, 
    dataset_text_field="text",
    args=training_args,
    tokenizer=tokenizer
)

trainer.train()


# ####################################
# STEP 5 Save the model
# ####################################

trainer.model.save_pretrained(output_dir)
trainer.tokenizer.save_pretrained(output_dir)

print(f"Modello fine-tunato salvato in {output_dir}")

wandb.finish()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,1.477000
20,1.139600
30,0.964000
40,0.897400
50,0.747100
60,0.728700
70,0.713800
80,0.733500
90,0.733200
100,0.747800


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Modello fine-tunato salvato in ./finetuned_tinyllama_lora


train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇█████
train/grad_norm,█▃▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,████▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁
train/loss,█▄▂▂▁▂▂▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁
total_flos,1.4935145454855782e+17
train/epoch,4.87734
train/global_step,1500
train/grad_norm,0.86986
train/learning_rate,0.0
train/loss,0.6551


# Evaluation of our second model (fine-tuning with LoRA)

In this case, we do two things, the first is to assess whether the model works correctly, i.e. whether it correctly generates the response
<br><br>
Next, we test the loss on the test set to see how the model performs and validate whether it has actually learnt in the training.<br>
For this part we use the trainer, which is a convenient and similar implementation to the training.

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


# ####################################
# STEP 1 Load model + LoRa + tokenizer
# ####################################

base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
adapter_path = "./finetuned_tinyllama_lora"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token  # Imposta il token di padding come EOS


# ####################################
# STEP 2 Generate a response in our format
# ####################################

prompt = "What is the best programming language for Machine Learning?"
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"

inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_k=50,
        top_p=0.7,
        temperature=0.7,
        repetition_penalty=1.1
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nGenerated response:")
print(generated_text)

Caricamento del modello di base...
Caricamento degli adattatori LoRA...
Caricamento del tokenizer...
Esempio di generazione del testo...

Risposta Generata:
<|im_start|>user
What is the best programming language for Machine Learning?<|im_end|>
<|im_start|>assistant
There are many programming languages used for machine learning, but the most common and popular choice is Python. Python is a high-level, versatile language that is known for its ease of use and widespread adoption in the AI industry. It has a large and active community of developers who develop and release new libraries and modules to further enhance the language's capabilities.

Other popular choices for machine learning programming languages include R, Java, C++, C#, and Scala.


In [34]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments
)
from peft import PeftModel
from trl import SFTTrainer
from datasets import Dataset



# ####################################
# STEP 1 Load model + LoRa + tokenizer
# ####################################

base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
adapter_path = "./finetuned_tinyllama_lora"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    torch_dtype=torch.float16, 
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token  



# ####################################
# STEP 2 make test in correct form
# ####################################

def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>\n", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])

test_dataset = format_data(df_test)



# ####################################
# STEP 3 Trainer using for evaluation
# ####################################

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    fp16=True, 
    report_to="none", 
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=512,  
)



# ####################################
# STEP 4 get the evaluation of our model
# ####################################

results = trainer.evaluate()
print(f"mean Loss on our test set: {results['eval_loss']}")


Caricamento del modello di base...
Caricamento degli adattatori LoRA...
Caricamento del tokenizer...
Configurazione del Trainer per la valutazione...


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Calcolo della loss sul test set...


Loss media sul test set: 1.8103272914886475
